## Project proposal
### Importing the dataset

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 16)
library(ggplot2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.4     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [1]:
studenturl<- "https://archive.ics.uci.edu/static/public/320/student+performance.zip"
temp <- tempfile()
download.file(studenturl,temp)
studentzip <- unzip(temp, "student.zip")
studentfile <- unzip(studentzip,"student-por.csv")
student <- read_csv2(studentfile, col_names = TRUE)

student

MethodError: MethodError: no method matching -(::String)
Closest candidates are:
  -(!Matched::Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8}) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:85
  -(!Matched::T, !Matched::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:86
  -(!Matched::Union{UInt16, UInt32, UInt64, UInt8}, !Matched::BigInt) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/gmp.jl:532
  ...

## Dataset background
## Possible questions from the dataset
- Proportion of students getting high to very high freetime after school in urban area in comparison of proportion of students getting high to very high freetime after school in rural area
- Mean final grade of students receiving extra support from school and family in comparison to students getting no support from school and family
- The mean number of absences for students in urban area in comparison to students in rural area. 
- Mean number of past class failures for students with family support in comparison to students with no family support. 3. mean final grade of students with a romantic relationship in comparison to the mean final grade of students with no romantic relationship.